In [1]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.callbacks import get_openai_callback


In [2]:
from langchain.tools import __all__ as public_api
print('\n'.join(public_api[10:25]) + '\n...')

AzureCogsSpeech2TextTool
AzureCogsText2SpeechTool
AzureCogsTextAnalyticsHealthTool
BaseGraphQLTool
BaseRequestsTool
BaseSQLDatabaseTool
BaseSparkSQLTool
BaseTool
BearlyInterpreterTool
BingSearchResults
BingSearchRun
BraveSearch
ClickTool
CopyFileTool
CurrentWebPageTool
...


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
groq_api_key = os.getenv("GROQ_API_KEY")

# MODEL_NAME = "mixtral-8x7b-32768"
# MODEL_NAME="llama3-groq-70b-8192-tool-use-preview"

# User the following for tool calling
MODEL_NAME="llama-3.1-70b-versatile" 

llm = ChatGroq(
    temperature=0,
    model_name=MODEL_NAME,
    groq_api_key=groq_api_key
)

In [5]:
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a cpecified city
    
    Args:
        city (str): The name of the city where you want to check the weather
        
    Returns:
        str: A description of the current weather in a specified city
    """
    return "Sunny, 33°C"


tools = [multiply, add, fake_weather_api]

In [6]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant skilled in math operations."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [7]:
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
result = agent_executor.invoke({"input": "What's 3 plus 5 multiplied by 2?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': '3', 'y': '10'}`


13.0
Invoking: `multiply` with `{'x': '5', 'y': '2'}`


10.0The answer is 13.

> Finished chain.
{'input': "What's 3 plus 5 multiplied by 2?", 'output': 'The answer is 13.'}


In [9]:
result = agent_executor.invoke({"input": "What's the current weather in Tel-Aviv?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `fake_weather_api` with `{'city': 'Tel-Aviv'}`


Sunny, 33°CThe current weather in Tel-Aviv is sunny with a temperature of 33°C.

> Finished chain.
{'input': "What's the current weather in Tel-Aviv?", 'output': 'The current weather in Tel-Aviv is sunny with a temperature of 33°C.'}


In [10]:
result

{'input': "What's the current weather in Tel-Aviv?",
 'output': 'The current weather in Tel-Aviv is sunny with a temperature of 33°C.'}

In [11]:
from langchain_core.callbacks.base import BaseCallbackHandler

class PromptPrintingCallbackHandler(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        print("\nFull Prompt:")
        for prompt in prompts:
            print(prompt)

# Use the custom callback handler
callback_handler = PromptPrintingCallbackHandler()

llm = ChatGroq(
    temperature=0,
    model_name=MODEL_NAME,
    groq_api_key=groq_api_key,
    callbacks=[callback_handler]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


result = agent_executor.invoke(
    {"input": "What's 3 plus 5 multiplied by 2?"}
)

print(result)



> Entering new AgentExecutor chain...

Full Prompt:
System: You're a helpful assistant skilled in math operations.
Human: What's 3 plus 5 multiplied by 2?

Invoking: `add` with `{'x': 3, 'y': 10}`


13.0
Invoking: `multiply` with `{'x': 5, 'y': 2}`


10.0
Full Prompt:
System: You're a helpful assistant skilled in math operations.
Human: What's 3 plus 5 multiplied by 2?
AI: 
Tool: 13.0
Tool: 10.0
The answer is 13.

> Finished chain.
{'input': "What's 3 plus 5 multiplied by 2?", 'output': 'The answer is 13.'}
